In [1]:
from vncorenlp import VnCoreNLP

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
def preTreatment(sentences, length):
    punctuations = []     # Dấu câu
    with open("punctuation.txt", "r", encoding = "utf-8") as data:
        for line in data:
            punctuations.append(line.strip())
    data.close()
    
    stopWords = []
    with open("vietnamese-stopwords-dash.txt", "r", encoding = "utf-8") as data:
        for line in data:
            stopWords.append(line.strip())
    data.close()
    
    for i in range(0, length):
        for j in range(0, len(sentences[i])):
            sentences[i][j] = sentences[i][j].lower()
        #print(sentences[i])
        
    # Loại bỏ dấu câu và stop words
    for i in range(0, length):
        sentences[i] = list(set(sentences[i]) - set(punctuations) - set(stopWords))
        #print(sentences[i])

In [5]:
def adjacentMatrix(sentences, length):
    # Tạo ma trận kề của đồ thị có trọng số
    matrix = np.zeros((length, length))
    for i in range(0, length):
        for j in range(0, length):
            if i != j:
                jaccard = len(set(sentences[i]) & set(sentences[j])) / len(set(sentences[i]) | set(sentences[j]))
                if jaccard != 0:
                    matrix[i][j] = jaccard
    #for i in range(0, length):
        #for j in range(0, length):
            #print(matrix[i][j], end="\t")
        #print()
    return matrix

In [6]:
def textRank(matrix, length):
    # sumOut là vector trong đó sumOut[i] là tổng trọng số của các link out của matrix[i]
    sumOut = np.zeros(length)
    for i in range(0, length):
        for j in range(0, length):
            if matrix[i][j] != 0:
                sumOut[i] += matrix[i][j]
    #print(sumOut)
    
    WS = np.ones(length)        # vector điểm của các đỉnh ở lần tính thứ n
    preWS = np.ones(length)     # vector điểm của các đỉnh ở lần tính thứ n - 1
    
    d = 0.85
    x = 0
    
    # Áp dụng thuật toán ranking trên đồ thị có trọng số
    while x < 100:
        for i in range(0, length):
            sum = 0
            for j in range(0, length):
                if matrix[i][j] != 0:
                    sum += (matrix[j][i] / sumOut[j]) * preWS[j]
            WS[i] = (1 - d) + d * sum
        for i in range(0, length):
            preWS[i] = WS[i]
        #print("WS lần thứ:", x)
        #print(WS)
        x += 1
    return WS

In [7]:
def sentencesExtraction(WS, word_segmented_text, length):
    # Tìm đỉnh có điểm cao nhất
    dictWS = {}
    for i in range(0, length):
        dictWS[WS[i]] = i
        
    mainSentences = []
    
    for i in sorted(dictWS.keys()):
        mainSentences.append(word_segmented_text[dictWS[i]])
    return mainSentences

In [8]:
def computeTF(wordDict, sentence):
    tfDict = {}
    wordsCount = len(sentence)
    for sentence, count in wordDict.items():
        tfDict[sentence] = count/float(wordsCount)
    return tfDict

In [9]:
def computeIDF(sentences):
    import math
    idfDict = {}
    N = len(sentences)
    
    idfDict = dict.fromkeys(sentences[0].keys(), 0)
    for s in sentences:
        for word, val in s.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
        
    return idfDict

In [10]:
def computeTFIDF(tfSentence, idfs):
    tfidf = {}
    for word, val in tfSentence.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [11]:
def keywordsExtractionByTFIDF(sentences):
    wordSet = set(sentences[0])
    for i in range(1, len(sentences)):
        wordSet = wordSet.union(set(sentences[i]))
    #print(wordSet)
    wordDict = []
    for i in range(0, len(sentences)):
        wordDict.append(dict.fromkeys(wordSet, 0))
    #print(wordDict)
    for i in range(0, len(sentences)):
        for w in sentences[i]:
            wordDict[i][w] += 1
    tfSentences = []
    for i in range(0, len(sentences)):
        tfSentences.append(computeTF(wordDict[i], sentences[i]))
    #print(tfSentences)
    idfs = computeIDF(wordDict)
    #print(idfs)
    tfidfSentences = []
    for i in range(0, len(sentences)):
        tfidfSentences.append(computeTFIDF(tfSentences[i], idfs))
    #print(tfidfSentences)
    
    df = pd.DataFrame(tfidfSentences)
    arr = {}
    wordSet = list(wordSet)
    n = (len(wordSet)*5)//100 +1
    for i in range(0, len(wordSet)):
        arr[wordSet[i]] = max(df[wordSet[i]])
    keyWords = dict(sorted(arr.items(), key=lambda item: item[1], reverse=True))
    print("Keywords:")
    for i in range(0, n):
        print(list(keyWords.keys())[i])
    #print(keyWords)
    

In [14]:
def main():
    annotator = VnCoreNLP(r"C:\Users\admin\Desktop\kit\VnCoreNLP\VnCoreNLP-1.1.1.jar", annotators="wseg,pos,parse", max_heap_size='-Xmx2g')
    
    data = open("text.txt", "r", encoding="utf-8")
    text = data.read()
    
    annotated_text = annotator.annotate(text)
    word_segmented_text = annotator.tokenize(text)
    
    sentences = word_segmented_text.copy()
    length = len(word_segmented_text)
    
    preTreatment(sentences, length)
    keywordsExtractionByTFIDF(sentences)
    matrix = adjacentMatrix(sentences, length)
    WS = textRank(matrix, length)
    mainSentences = sentencesExtraction(WS, word_segmented_text, length)
    s = " "
    print(s.join(mainSentences[len(mainSentences) - 1]))

In [15]:
main()

Keywords:
đối_đáp
đại_diện
17/4
sửa_chữa
trở_lại
xã_hội
gia_đình
phiên_toà
nhận_thức
luật_sư lê_trung_sơn phân_tích thêm , khi cho ý_kiến đồng_ý với nội_dung tờ_trình số 835 , bị_cáo hưng chỉ có mong_muốn đổi_mới , tháo_gỡ khó_khăn cho dự_án .
